In [1]:
import os 
os.chdir("../")
%pwd

'd:\\Projects\\Phishing_Detector'

In [2]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelTrainerConfig:
    root_dir: Path
    train_data_path : Path
    test_data_path :Path
    model_name: str
    target_column: str

In [3]:
from phishing.constants import *
from phishing.utils.common import read_yaml,create_directories

In [4]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH,
                 schema_filepath = SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path=config.train_data_path,
            test_data_path=config.test_data_path,
            model_name=config.model_name,
            target_column=schema.name
        )
        return model_trainer_config

In [10]:
import pandas as pd 
import os 
from phishing.logging.custom_logger import logging
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
import joblib

In [ ]:
class ModelTrainer:
    def __init__(self,config= ModelTrainerConfig):
        self.config = config

    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        train_x = train_data.drop([self.config.target_column],axis=1)
        test_x = test_data.drop([self.config.target_column],axis=1)
        train_y = train_data[[self.config.target_column]].replace(-1,0)
        test_y = test_data[[self.config.target_column]].replace(-1,0)

        lr = XGBClassifier()
        lr.fit(train_x,train_y)

        joblib.dump(lr,os.path.join(self.config.root_dir,self.config.model_name))
    

In [11]:
try:
    config = ConfigurationManager()
    model_train_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(model_train_config)
    model_trainer.train()
except Exception as e:
    raise e 

0.9706946454413893
